In [1]:
import pandas as pd, numpy as np, scipy, os, itertools, sys
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
sys.path.append('..')
from func import loaders
from import_library import *

In [2]:
outcome_df = pd.read_csv('../data/clinical data/clinical_data_2.csv')[['subj', 'AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']]
data_type ='var' #mean, var
ld_excel = loaders(data_type = data_type)
ld_mat = loaders(data_type=data_type)
raw_data_path = '/Volumes/DatasetCGRD/SCD/data/image data/dFC matrix matlab/'
for next_path in sorted(os.listdir(raw_data_path)):
    if 'C0018FC_z.mat' not in next_path:
        ld_mat.load_mat(data_path = os.path.join(raw_data_path,next_path))
alff_index, alff_value, alff_columns = ld_excel.load_csv(data_path=f'../data/mdALFF matrix excel/mdALFF_{data_type}.csv')
fc_value = ld_mat.callback()

In [3]:
model_stack_ALFF = [None, None]
model_stack_FC = [None, None]
model_feature_stack = []
for idx, outcome_name in enumerate(['AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']):
# for idx, outcome_name in enumerate(['AD8主觀認知障礙[0-1,>=2]']):
    forest = RandomForestRegressor(max_depth = 5, n_jobs=-1, 
                        random_state=123)
    # FC feature selection
    print(f"FC {data_type} Feature Selection:", outcome_name)
    print(outcome_name)
    feat_selector_fc = BorutaPy(forest, verbose=0, random_state=123, max_iter=10)
    feat_selector_fc.fit(fc_value, np.array(outcome_df[outcome_name]))
    model_stack_FC[idx] = (feat_selector_fc)
    ranking_stack = []
    for (value, colum) in zip(feat_selector_fc.ranking_, [f'FC_{data_type}_{i}' for i in range (fc_value.shape[1])]):
        ranking_stack.append([value, colum])
    # ranking_stack = sorted(ranking_stack)
    pd_rs = pd.DataFrame(ranking_stack, columns=['rank', 'feature_location'])
    pd_rs.to_csv(f'./dataset/{outcome_name}_{data_type}_fc.csv', index=False)
    model_feature_stack.append(pd_rs)

    # # ALFF feature selection
    # print(f"ALFF {data_type} Feature Selection:", outcome_name)
    # feat_selector_alff = BorutaPy(forest, verbose=0, random_state=123, max_iter=10, alpha=0.01, two_step=True)
    # feat_selector_alff.fit(np.array(alff_value), np.array(outcome_df[outcome_name]))
    # model_stack_ALFF[idx] = feat_selector_alff

    # ranking_stack = []
    # for (value, colum) in zip(feat_selector_alff.ranking_,alff_value.columns.tolist()):
    #     ranking_stack.append([value, colum])
    # ranking_stack = sorted(ranking_stack)
    # pd_rs = pd.DataFrame(ranking_stack, columns=['rank', 'feature_location'])
    # pd_rs.to_csv(f'./dataset/{outcome_name}_{data_type}_overall.csv', index=False)
    # model_feature_stack.append(pd_rs)

ALFF var Feature Selection: AD8主觀認知障礙[0-1,>=2]
ALFF var Feature Selection: MOCA客觀認知測驗分數[<=23,>=24]


In [4]:
for i in zip(model_feature_stack, ['AD8主觀認知障礙[0-1,>=2]', 'MOCA客觀認知測驗分數[<=23,>=24]']):
    print(i[1], data_type)
    print(i[0].head(30))
    print('\n')

AD8主觀認知障礙[0-1,>=2] var
    rank       feature_location
0      2             164_Visual
1      2  253_SomatomotorDorsal
2      2            75_Auditory
3      3             149_Visual
4      4   211_VentralAttention
5      5     177_FrontoParietal
6      6             159_Visual
7      7  46_SomatomotorLateral
8      8        135_DefaultMode
9     10             171_Visual
10    10        277_DefaultMode
11    10         91_DefaultMode
12    12   29_SomatomotorDorsal
13    13   35_SomatomotorDorsal
14    14   260_CinguloOpercular
15    15        131_DefaultMode
16    16             161_Visual
17    16            76_Auditory
18    18             146_Visual
19    18   257_VentralAttention
20    20  300_SomatomotorDorsal
21    21   267_CinguloOpercular
22    22        120_DefaultMode
23    22             157_Visual
24    24             247_Reward
25    25             156_Visual
26    26            73_Auditory
27    26         95_DefaultMode
28    28           248_Salience
29    28   30_Som